# CATE & Double Machine Learning

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* CATEs - Conditional Average Treatment Effects (Efeitos Heterogêneos do Tratamento)
* Efeitos Heterogêneos do Tratamento e o Arcabouço de Resultados Potenciais
* Meta-learners
* Ortogonalização
  * Aplicação do Procedimento de Ortogonalização no Python
* Ortogonalização e Machine Learning  (Orthogonal/Double Machine Learning - DML)

## Referências

**Principais:**
* UBER CausalML: https://causalml.readthedocs.io/en/latest/
* Microsoft EconML: https://econml.azurewebsites.net/
* https://github.com/DoubleML/doubleml-for-py
* https://docs.doubleml.org/stable/index.html
* https://github.com/MasaAsami/ReproducingDMLDiD/blob/main/notebook/Reproduction_of_DMLDiD_RO_for_NEW_SIMDATA.ipynb
* https://econml.azurewebsites.net/
* Schmidheiny, K., & Siegloch, S. (2023). On event studies and distributed-lags in two-way fixed effects models: Identification, equivalence, and generalization. Journal of Applied Econometrics, 1- 19. https://doi.org/10.1002/jae.2971
* Stevenson, Betsey, Wolfers, Justin, 2006. Bargaining in the shadow of the law: Divorce laws and family distress. Q. J. Econ. 121 (1), 267–288.
* Goodman-Bacon, A. (2021). Difference-in-differences with variation in treatment timing. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2021.03.014
* Callaway, B. and Sant'Anna, P. H. C. (2021). Difference-in-Differences with multiple time periods. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2020.12.001

**Complementares:**

* Borusyak, K.; Jaravel, X. and Spiess, J. (2023). Revisiting Event Study Designs: Robust and Efficient Estimation. arXiv: https://arxiv.org/pdf/2108.12419.pdf
* Clément deChaisemartin, Xavier d’Haultfoeuille. (2022) Difference-in-Differences Estimators of Intertemporal Treatment Effects. hal-03873903
* Roth et al. (2022) What’s Trending in Difference-in-Differences? A Synthesis of the Recent Econometrics Literature. https://www.jonathandroth.com/assets/files/DiD_Review_Paper.pdf

## CATEs - Conditional Average Treatment Effects (Efeitos Heterogêneos do Tratamento)

A nossa função de expectativa condicional é dada por: 

$$ E[Y|X, T]$$

Suponha que queremos realizar inferência causal, entre $T$ e $Y$, sob um contexto $X$, e otimizá-la. Ou seja, 

$$ \text{argmax}_{T} E[Y|X, T]$$

Ou seja, agora queremos mais do que apenas o efeito médio do tratamento (ATE). Sabemos que o tratamento tem impacto positivo em algumas pessoas, mas não em outras. Os recursos $X$ (covariáveis) desempenham um papel na definição de diferentes perfis de unidades (indivíduos), e cada perfil (indivíduo) **pode responder de forma diferente ao tratamento**. Sendo assim, **agora queremos personalizar o tratamento, dando-o apenas às unidades que melhor respondem a ele** (a ideia é "direcionar o tratamento"). Por exemplo, se um medicamento tem efeitos colaterais graves para crianças, podemos querer restringir sua distribuição apenas para adultos. Ou se uma campanha publicitária é eficaz apenas em países de língua inglesa, não vale a pena mostrá-la em outro lugar.

**CATE**: 

O **Conditional Average Treatment Effect** (CATE) é a média do efeito do tratamento condicional a um conjunto de características. Para o caso de tratamento binário, temos: 

$$ E[Y_{1}-Y_{0}|X]$$

ou, para o caso de tratamento contínuo:

$$ E[y´(t)|X]$$

O condicionamento em $X$ significa que agora permitimos que o efeito do tratamento seja diferente dependendo das características de cada unidade (indivíduo). Queremos tratar apenas as unidades certas (no caso binário) ou descobrir qual é a dosagem de tratamento ideal para cada unidade (no caso contínuo).

Considere o exemplo gráfico:

<div style="text-align:center;">
    <img src="images\elast-partition.png"  alt="Imagem" style="width: 400px;"/>
</div>

Desagregando os dados, podemos ver que o efeito do tratamento é diferente para diferentes os 3 grupos de pessoas. 

<div style="text-align:center;">
    <img src="images\elast-split.png"  alt="Imagem" style="width: 800px;"/>
</div>

Assim, a ideia é buscar a indetificação do efeito heterogêneo do tratamento, para que possamos personalizar o tratamento para cada unidade. Vamos tentar entender esse mesmo gráfico de outra maneira. Suponha que queremos encontrar os dias em que a $ \frac{d \text{Vendas}}{d \text{Preço}} $ é menor. Isso significa que, para esses dias, a elasticidade-preço é menor. Logo, do ponto de vista do vendedor, poderíamos aumentar o preço nesses dias sem perder muitas vendas (será que isso ocorre no mercado de aluguéis de imóveis durante às férias?).

A elasticidade (sensibilidade do indivíduo) é não observável. Podemos pensar cada unidade como tendo um valor $Y_{i}$, com uma elasticidade (sensibilidade) individual ($ \frac{d Y_{i}}{d T} $). 

<div style="text-align:center;">
    <img src="images\elasticity.png"  alt="Imagem" style="width: 400px;"/>
</div>


Para vermos as inclinações individuais, teríamos que observar cada dia sob dois preços diferentes e calcular como as vendas mudam para cada um desses preços. 

$$ \frac{dY_{i}}{dT} = \frac{Y_{i}(T_{i}) - Y_{i}(T_{i} + \epsilon)}{T_{i}-(T_{i} + \epsilon)}$$

Este é o problema fundamental da inferência causal novamente. Nunca podemos ver a mesma unidade sob diferentes condições de tratamento. Então, o que podemos fazer?


Um possível ajuste aos dados é utilizar o OLS.

$$ y_{i} = \alpha + \beta T_{i} + \gamma X_{i} + \epsilon_{i} $$

Diferenciando o tratamento,

$$ \frac{dY_{i}}{dT} = \beta$$

Nesse caso, é um modelo simples, e um valor constante de beta para todos os indivíduos. Esse é o ATE. No entento, se fizermos a seguinte mudança simples:

$$ y_{i} = \alpha + \beta T_{i} + \gamma X_{i} + \delta X_{i}T_{i} + \epsilon_{i} $$

Teremos uma sensibilidade diferente para cada indivíduo:

$$ \frac{dY_{i}}{dT} = \beta + \delta X_{i}$$

Onde $\delta$ é um coeficiente que depende das características de cada indivíduo ($X_{i}$). Em outras palavras, a previsão de sensibilidade mudará conforme $X$. Com essas previsões de sensibilidade, podemos agrupar as unidades por quanto achamos que elas responderão ao tratamento.






## Efeitos Heterogêneos do Tratamento e o Arcabouço de Resultados Potenciais

Podemos definir o efeito do tratamento individual (Individual Treatment Effect - ITE - $\beta_{i}$) como a diferença entre os resultados potenciais. 

$$ \beta_{i}^{ITE} = Y_{i}(1) - Y_{i}(0)$$

ou, no caso do tratamento contínuo, $\beta_{i}^{ITE} = dY_{i}/dt$, onde t é a variável de tratamento. 

Segundo o problema fundamental da inferência causal, nunca podemos observar o mesmo indivíduo sob diferentes condições de tratamento. 

$$
Y^{obs}_i(t)= 
\begin{cases}
Y_i(1), & \text{se } t=1\\
Y_i(0), & \text{se } t=0
\end{cases}
$$


Podemos definir o efeito médio do tratamento (Average Treatment Effect - ATE) como


$$
\beta^{ATE}= E[Y_i(1) − Y_i(0)] = E[\beta_i]
$$

e o efeito do tratamento médio condicional (Conditional Average Treatment Effect - CATE) como


$$
\beta^{CATE}(x) = E[Y_i(1) − Y_i(0)|X] = E[\beta_i|X_{i}=x]
$$

Os ITE são inerentemente não observáveis. O que pode ser estimado em vez disso é o Conditional Average Treatment Effect (CATE) , ou seja, o efeito esperado do tratamento individual, condicional em covariáveis ​​X.

Para recuperar o CATE, precisamo fazer 3 suposições:

* Inconfundibilidade (Unconfoundedness): $Y_{i}(0), Y_{i}(1) \perp T|X$
* Sobreposição (Overlap): $0 < p(x) < 1$
* Consistência (Consistency): $Y_{i} = Y_{i}(1)T_{i} + Y_{i}(0)(1-T_{i})$

Onde $p(x)$ é o escore de propensão, ou seja, a probabilidade esperada de ser tratado, condicional às covariáveis ​​$X$.

Cabe destacar que os modelos lineares têm algumas desvantagens. A principal delas é a suposição de linearidade em $X$. Seria ótimo se pudéssemos substituir o modelo linear por um modelo de machine learning mais flexível. Poderíamos até mesmo conectar o tratamento como um recurso a um modelo de ML, como Decision Trees, rede neural ou Gradient Boosting. 

$$ y_{i} = M(X_{i}, T_{i}) + \epsilon_{i}$$


mas a partir daí, não está claro como podemos obter estimativas do efeito do tratamento, uma vez que este modelo produzirá previsões de $Y$, e não de $\beta$. Vamos aprender uns conceitos importantes para solucionar essa questão.





## Meta-learners

Podemos usar métodos de aprendizado de máquina para estimar de forma flexível efeitos de tratamento heterogêneos. Em particular, vamos inspecionar três métodos populares que foram introduzidos por Künzel, Sekhon, Bickel, Yu, (2019), https://www.pnas.org/doi/abs/10.1073/pnas.1804597116 

Contexto de análise:

Assumimos que para um conjunto de indivíduos iid, e observamos uma ($X_{i}, T_{i}, Y_{i}$) composto de

* Tratamento $T_{i} \in \{0,1\}$
* Recursos $X_{i} \in \mathbb{R}^{n}$
* Resposta $Y_{i} \in \mathbb{R}$

Estamos interessados ​​em estimar o efeito médio do tratamento.

$$ \beta= E[Y_{i}^{(1)} - Y_{i}^{(0)}]$$

Onde $Y_{i}^{(1)}$ e $Y_{i}^{(0)}$ são os resultados potenciais de um indivíduo sob tratamento e controle, respectivamente.

Hipóteses:

* **Inconfundibilidade (Unconfoundedness)**: $Y_{i}(0), Y_{i}(1) \perp T|X$. Ou seja, condicional a características observáveis $X$, a atribuição do tratamento $T$ é tão bom quanto uma aleatorização. O que estamos efetivamente assumindo é que não há outras características que não observamos que poderiam impactar tanto T quanto Y. Esta é uma forte suposição que tem mais probabilidade de ser satisfeita quanto mais características individuais observamos.
* **valor de tratamento unitário estável (SUTVA)**: $Y_{i}(t) \perp T $. Ou seja, o resultado potencial não depende do status do tratamento. 

**S-learner** (Single-learner)

O meta-algoritmo mais simples é o single learner ou S-learner . Para construir o estimador S-learner, ajustamos um único modelo para todas as observações.

$$ \mu(z) = E[Y_{i}| (X_{i}, T_{i}) = z]$$

O estimador é dado pela diferença entre os valores previstos avaliados com e sem o tratamento.

$$ \hat{\beta}_{S-learner}(x) =  \mu(x, 1) - \mu(x, 0) $$

O problema com o S-learner é que ele está aprendendo um único modelo, então temos que esperar que o modelo descubra heterogeneidade no tratamento $T$, mas pode não ser o caso. Além disso, se o modelo for fortemente regularizado devido à alta dimensionalidade de $X$, pode não recuperar nenhum efeito de tratamento.

**T-learner** (Two-learner)

Para construir o estimador de dois alunos/aprendizes ou T-learner, ajustamos dois modelos diferentes, um para unidades tratadas e outro para unidades de controle.

$$ \mu_{1}(x) = E[Y_{i}| X_{i}= x, T_{i}=1 ]$$

$$ \mu_{0}(x) = E[Y_{i}| X_{i}= x, T_{i}=0 ]$$

O estimador é dado pela diferença entre os valores previstos avaliados com e sem o tratamento.

$$ \hat{\beta}_{T-learner}(x) =  \mu_{1}(x) - \mu_{0}(x) $$

o T-learner é muito mais flexível do que o S-learner porque ele se encaixa em dois modelos separados. O problema agora é que estamos usando apenas uma fração dos dados para cada problema de previsão, enquanto o S-learner estava usando todos os dados. Ao ajustar dois modelos separados, estamos perdendo algumas informações. Além disso, ao usar dois modelos diferentes, podemos obter heterogeneidade onde não há nenhuma .

**X-learner**

O estimador cross-learner ou X-learner é uma extensão do estimador T-learner. Ele é construído da seguinte maneira:

* Quanto ao T-learner, calcule modelos separados para $\mu_{1}(x)$ e $\mu_{0}(x)$, utilizando as unidades tratadas e de controle, respectivamente
* Calcule os efeitos estimados do tratamento como:
  
$$
\Delta_{i}(x) =
\begin{cases}
Y_{i} - \hat{\mu}_{0}(x) & \text{se } T_{i}=1 \\
\hat{\mu}_{1}(x) - Y_{i} & \text{se } T_{i}=0
\end{cases}
$$

   * Prevendo $\Delta$ de $X$, calcular $\hat{\beta}^{0}(x)$ de unidades tratadas, e $\hat{\beta}^{1}(x)$ de unidades de controle.
   * Estimar o escore de propensão

$$ p(x) = P(T=1|X=x) = E[T|X_{i}=x]$$

   * Calcular os efeitos do tratamento

$$ \hat{\beta}_{X-learner}(x) = p(x) \hat{\beta}^{0}(x) + (1-p(x)) \hat{\beta}^{1}(x)$$


Assim é possivel recuperar as funções de pseudo-resposta. Podemos reescrever os efeitos do tratamento como:

$$ \hat{\beta}_{X-learner}(x) = \hat{\beta}^{0}(x).\hat{p}(x) + \hat{\beta}^{1}(x).(1-\hat{p}(x)) = \hat{p}(x)[\hat{\mu}^{1}(x) - Y_{i}^{0}] + (1-\hat{p}(x))[Y_{i}^{1} - \hat{\mu}^{0}(x)] = [\hat{p}(x)\hat{\mu}^{1}(x) + (1-\hat{p}(x))Y_{i}^{1}] - [\hat{p}(x)Y_{i}^{0} + (1-\hat{p}(x))\hat{\mu}^{0}(x)] $$

Para que as funções de pseudo-resposta estimadas pelo X-learner sejam

$$ \tilde{\mu}^{1}(x) = \hat{p}(x)\hat{\mu}^{1}(x) + (1-\hat{p}(x))Y_{i}^{1}$$
$$ \tilde{\mu}^{0}(x) = \hat{p}(x)Y_{i}^{0} + (1-\hat{p}(x))\hat{\mu}^{0}(x)$$


Como podemos ver, o X-learner combina os verdadeiros valores $Y_{i}^{(d)}$ com os estimados $\mu_{i}^{(d)(x)}$ ponderado pelos escores de propensão $p(x)$, ou seja, as probabilidades de tratamento estimadas.

O que isso significa? Significa que se tivermos muito mais observações para um grupo (no nosso caso, o grupo de controle), a função de resposta de controle $\hat{\mu}^{0}(x)$ obterá a maior parte do peso. Em vez disso, para o outro grupo (o grupo de tratamento em nosso caso), as observações reais $Y_{i}^{1}$ receberá a maior parte do peso.

A principal vantagem do X-learners é que ele adapta a flexibilidade das funções de resposta ao contexto. Em áreas do espaço de estado onde temos muitos dados, ele usa principalmente a função de resposta estimada, em áreas do espaço de estado com poucos dados, ele usa as próprias observações.

* Considerações sobre os Learners/Aprendizes: 
  * A coisa mais simples que podemos fazer é usar um único ou S-learner com o tratamento como um recurso. Isso tende a funcionar bem quando o tratamento não é um preditor fraco do resultado. Mas se esse não for o caso, o S-learner tende a ser tendencioso para zero ou até mesmo abandonar o tratamento completamente. Adicionando um pouco mais de complexidade, podemos forçar o aluno a pegar o tratamento usando um T-learner. Aqui, ajustamos um modelo de Machine Learning por nível de tratamento. Isso funciona bem quando há amostras suficientes para todos os níveis de tratamento, mas pode falhar quando um nível de tratamento tem um tamanho de amostra pequeno, forçando um modelo a ser fortemente regularizado. Para consertar isso, podemos adicionar outro nível de complexidade usando um X-learner, onde temos dois estágios de ajuste e usamos um modelo de pontuação de propensão para corrigir erros potenciais de modelos estimados com muito poucos pontos de dados. Um grande problema desses aprendizes (exceto o aprendiz S) é que eles assumem um tratamento binário ou categórico. 

Há um aprendiz adicional que ainda não vimos, que é mais geral: **o aprendiz R**. Antes, precisamos aprender mais um conceito.


## Ortogonalização

Antes de estimar o CATE, precisamos conhecer um conceito importante: a Ortogonalização, e como ela é aplicada na econometria. A ideia de ortogonalização é baseada em um teorema elaborado por três econometristas em 1933, Ragnar Frisch, Frederick V. Waugh e Michael C. Lovell. Simplificando, afirma que você pode decompor qualquer modelo de regressão linear multivariável em três estágios ou modelos. 

Digamos que você tem uma matriz de covariáveis X, e voce particiona ela em duas partes, $X_{1}$ e $D$. 

* Primeira etapa
Pegamos o primeiro conjunto de variáveis $X_{1}$ e fazemos uma regressão linear de $X_{1}$ em $Y$, onde $\theta_{1}$ é o vetor de parâmetros

$$ y_{i} = \theta_{0} + \theta_{1} X_{1i} + \epsilon_{i}$$

e guardamos os resíduos dessa regressão ($y^{*}$).

$$ y^{*}_{i} = y_{i} - \hat{y}_{i} = y_{i} - ( \hat{\theta}_{0} + \hat{\theta}_{1} X_{1i} )$$

* Segunda etapa

Pegamos novamente o primeiro conjunto de características, mas agora executamos um modelo onde estimamos o segundo conjunto de características ($X_{2}$)

$$ D_{i} = \gamma(0) + \gamma(1) X_{1i} + e_{i}$$

Aqui, estamos usando o primeiro conjunto de recursos para prever o segundo conjunto de recursos. Por fim, consideramos também os resíduos desta segunda etapa.

$$ D_{i}^{*} = D_{i} - (\gamma(0) + \gamma(1) X_{1i})$$

* Terceira etapa
Por fim, pegamos os resíduos do primeiro e do segundo estágio e estimamos o seguinte modelo

$$ y_{i}^{*} = \beta_{0} + \beta_{2} D_{i}^{*} + e_{i}$$


* Teorema Frisch – Waugh – Lovell (FWL)

O teorema FWL afirma que a estimativa do parâmetro $\hat{\beta}_{2}$, estimado anteriormente, é equivalente ao que obtemos ao executar a regressão completa, com todas as covariáveis.

$$ y_{i} = \beta_{0} + \beta_{1} X_{1i} + \beta_{2} D_{i} + e_{i}$$

**Intuição**

Sabemos que a regressão é um modelo muito especial. Cada um de seus parâmetros tem a interpretação de uma derivada parcial, quanto seria Y se X aumentasse em uma unidade, mantendo todos as outras covariáveis constantes. Sabemos também que se omitirmos variáveis ​​da regressão, teremos viés. Especificamente, viés variável omitido (ou viés de confusão). Ainda assim, Frisch-Waugh-Lovell está dizendo que posso dividir meu modelo de regressão em duas partes, nenhuma delas contendo o conjunto completo de recursos, e ainda assim obter a mesma estimativa que obteria executando a regressão inteira. 

O teorema fornece algumas dicas sobre o que a regressão linear está fazendo. Para obter o coeficiente de uma variável $X_{k}$, a regressão primeiro usa todas as outras variáveis ​​para prever $X_{k}$ e pega os resíduos. Isso “limpa” $X_{k}$ de qualquer influência dessas variáveis. Dessa forma, quando tentamos entender o impacto de $X_{k}$ sobre $Y$, estará livre de viés de variável omitida. Em segundo lugar, a regressão usa todas as outras variáveis ​​para prever $Y$ e pega os resíduos. Isso “limpa” $Y$ de qualquer influência dessas variáveis, reduzindo a variância de $Y$ para que seja mais fácil ver como $X_{k}$ afeta $Y$.

A regressão linear está estimando o impacto de $D$ sobre $y$ enquanto contabiliza $X_{1}$. Isso é importante para inferência causal. 

Assim, podemos construir um modelo que preveja um tratamento ($T$) usando as covariáveis $X$, um modelo que prevê o resultado $y$ usando as mesmas covariáveis, pega os resíduos de ambos os modelos e executa um modelo que estima como o resíduo de $T$ afeta o resíduo de $y$. Este último modelo vai me dizer como $T$ afeta $y$ enquanto controla por $X$. Ou seja, os dois primeiros modelos controlam as variáveis de confusão. Eles estão gerando dados que são praticamente aleatórios. Isso está distorcendo meus dados. É isso que usamos no modelo final para estimar a elasticidade.

### Aplicação do Procedimento de Ortogonalização no Python


In [56]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [57]:
# DataFrame
df = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")

# Criar a variável de resultado
df['Y'] = df['bweight']

# Crie a variável 'Treated' com valor inicial de 0
df['Treated'] = 0
# Recodifique 'Treated' para 1 se 'mbsmoke' for igual a 'smoker'
df.loc[df['mbsmoke'] == 'smoker', 'Treated'] = 1

df['casada'] = 0
df.loc[df['mmarried']=='married', 'casada'] = 1


Para desviar este conjunto de dados, precisaremos de dois modelos. O primeiro modelo, vamos chamá-lo $M_{t}(X)$, prevê o tratamento (Se a gestante é fumante, no nosso caso) utilizando os confundidores. É um dos estágios que vimos acima, no teorema de Frisch–Waugh–Lovell.

In [58]:
m_D = smf.ols("Treated ~ 1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
df['m_D_star'] = df['Treated'] - m_D.predict(df)

Assim que tivermos este modelo, construiremos os resíduos

$$ \hat{D}_{i} = D_{i} - M_{t}(X_{i})$$

Você pode pensar neste resíduo como uma versão do tratamento que é imparcial ou, melhor ainda, que é impossível de prever a partir dos fatores de confusão $X$. Como os fatores de confusão já eram usados ​​para prever $t$, o resíduo é, por definição, imprevisível com com $X$. Outra maneira de dizer isso é que o viés foi explicado pelo modelo $M_{t}(X)$, produzindo $\hat{t}_{i}$ que é tão bom quanto atribuído aleatoriamente. É claro que isso só funciona se tivermos em $X$ todos os fatores de confusão que causam ambos $T$ e $Y$.

Também podemos construir resíduos para o resultado.

$$ \hat{y}_{i} = y_{i} - M_{y}(X_{i})$$


Este é outro estágio do teorema de Frisch – Waugh – Lovell. Isso não torna o conjunto menos tendencioso, mas facilita a estimativa do efeito, reduzindo a variância em $y$. Mais uma vez você pode pensar $\hat{y}_{i}$ como uma versão de $y_{i}$ imprevisível de $X$ ou que teve todas as suas variações devido a $X$ explicadas. Pense nisso. Nós já usamos $X$ para prever $y$ com $M_{y}(X_{i})$. E $\hat{y}_{i}$ é o erro dessa previsão. Então, por definição, não é possível prever isso a partir de $X$. Todas as informações em $X$ para prever $y$ já foram usadas. Se for esse o caso, a única coisa que resta para explicar $\hat{y}_{i}$ é algo que não usamos usamos para construí-lo (não incluído em $X$), que é apenas o tratamento (novamente, assumindo que não há fatores de confusão não medidos).


In [59]:
m_y = smf.ols("Y ~  1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
df['m_y_star'] = df['Y'] - m_y.predict(df)

Considerando o OLS tradicional com covariáveis.

In [60]:
ols = smf.ols("Y ~ Treated + 1 + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu", data=df).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     38.49
Date:                qua, 14 ago 2024   Prob (F-statistic):          5.80e-100
Time:                        17:17:13   Log-Likelihood:                -35858.
No. Observations:                4642   AIC:                         7.175e+04
Df Residuals:                    4627   BIC:                         7.184e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2851.5203     54.983     51.861      0.0

Agora vamos verificar o teorema de Frisch-Waugh-Lovell.

In [61]:
FWL1 = smf.ols("m_y_star ~ m_D_star", data=df).fit()
print(FWL1.summary())

                            OLS Regression Results                            
Dep. Variable:               m_y_star   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     100.6
Date:                qua, 14 ago 2024   Prob (F-statistic):           1.91e-23
Time:                        17:17:17   Log-Likelihood:                -35858.
No. Observations:                4642   AIC:                         7.172e+04
Df Residuals:                    4640   BIC:                         7.173e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.348e-11      8.041   2.92e-12      1.0

Depois de fazermos as duas transformações, a única coisa que resta para prever esses resíduos é o tratamento. 

Para resumir, ao prever o tratamento, construímos $\hat{t}$ que funciona como uma versão imparcial do tratamento; ao prever o resultado, construímos $\hat{y}$ que é uma versão do resultado que só pode ser explicada se usarmos o tratamento. Esses dados, onde substituímos por $y$ por $\hat{y}$ e $t$ por $\hat{t}$, são os dados desviados que queríamos. Podemos usá-lo para avaliar nosso modelo causal da mesma forma que fizemos anteriormente, usando dados aleatórios.

###  DML - Orthogonal/Double Machine Learning

Quando temos muitas variáveis ​​de controle, podemos querer selecionar as mais relevantes , possivelmente capturando não linearidades e interações. Algoritmos de aprendizado de máquina são perfeitos para essa tarefa. No entanto, nesses casos, estamos introduzindo um viés que é chamado de regularização ou pré-teste, ou viés de seleção de recursos. 

Ou seja, o que acontece se a dimensão de X aumenta e não conhecemos a formafuncional através da qual X afeta Y e D? 

Nesses casos, podemos usar algoritmos de aprendizado de máquina para descobrir essas relações não lineares de alta dimensão.

No artigo de **Chernozhukov et al (2016)**, os autores mostraram que também é possível fazer ortogonalização com modelos de aprendizado de máquina. 

$$ \hat{y}_{i} = y_{i} - M_{y}(X_{i})$$

$$ \hat{D}_{i} = D_{i} - M_{t}(X_{i})$$

Os modelos de aprendizado de máquina podem ajustar-se perfeitamente aos dados, ou melhor, superajustá-los (overfitting). 

* Apenas olhando para as equações acima, podemos saber o que acontecerá nesse caso. Se $M_{y}$ de alguma forma, os resíduos serão todos muito próximos de zero. Se isso acontecer, será difícil descobrir como $t$ afeta isso. 
* Da mesma forma, se $M_{t}$ de alguma forma superajusta, seus resíduos também serão próximos de zero. Conseqüentemente, não haverá variação no resíduo do tratamento para ver como isso pode impactar o resultado.

Para explicar isso, precisamos fazer a divisão da amostra. Ou seja, estimamos o modelo com uma parte do conjunto de dados e fazemos previsões na outra parte. A maneira mais simples de fazer isso é dividir a amostra de teste ao meio, fazer dois modelos de forma que cada um seja estimado em uma metade do conjunto de dados e faça previsões na outra metade.

Uma implementação um pouco mais elegante usa **validação cruzada K-fold**. A vantagem é que podemos treinar todos os modelos em uma amostra maior que metade do conjunto de teste.

<div style="text-align:center;">
    <img src="images\kfold-cv.png"  alt="Imagem" style="width: 300px;"/>
</div>

Felizmente, esse tipo de previsão cruzada é muito fácil de implementar usando `cross_val_predicta` função do Sklearn.


Vamos utilizar o Random Forest para fazer a previsão do tratamento e do resultado.

In [49]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor

X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"

folds = 5

np.random.seed(123)
m_D = RandomForestRegressor(n_estimators=100)
D_res1 = df[D] - cross_val_predict(m_D, df[X], df[D], cv=folds)

m_y = RandomForestRegressor(n_estimators=100)
y_res1 = df[y] - cross_val_predict(m_y, df[X], df[y], cv=folds)

Agora que temos os resíduos, vamos armazená-los como colunas em um novo conjunto de dados.

In [50]:
DML1 = df.assign(**{
    "Y-ML_y(X)": y_res1,
    "Treated-ML_t(X)": D_res1,
})
DML1.head()

,bweight,mmarried,mhisp,fhisp,foreign,alcohol,deadkids,mage,medu,fage,...,fbaby,prenatal1,Y,Treated,casada,m_t_star,m_y_star,m_D_star,Y-ML_y(X),Treated-ML_t(X)
0,3459,married,0,0,0,0,0,24,14,28,...,No,Yes,3459,0,1,-0.086026,47.203835,-0.086026,-295.130000,-0.026667
1,3260,notmarried,0,0,1,0,0,20,10,0,...,No,Yes,3260,0,0,-0.315814,277.930277,-0.315814,303.140000,-0.410000
2,3572,married,0,0,1,0,0,22,9,30,...,No,Yes,3572,0,1,-0.186338,196.895284,-0.186338,132.310000,-0.070000
3,2948,married,0,0,0,0,0,26,12,30,...,No,Yes,2948,0,1,-0.156887,-451.977522,-0.156887,-460.669000,0.000000
4,2410,married,0,0,0,0,0,20,12,21,...,Yes,Yes,2410,0,1,-0.121311,-1040.633348,-0.121311,-1350.973333,0.000000


In [54]:
FWL_DML1 = smf.ols("y_res1 ~ D_res1", data=DML1).fit()
print(FWL_DML1.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_res1   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     66.82
Date:                qua, 14 ago 2024   Prob (F-statistic):           3.80e-16
Time:                        17:08:49   Log-Likelihood:                -36189.
No. Observations:                4642   AIC:                         7.238e+04
Df Residuals:                    4640   BIC:                         7.239e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.1388      8.646      0.479      0.6

Vejamos outro exemplo do DML. Considerando Gradient Boosting Machines (GBM) para prever o tratamento e o resultado.

In [52]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor

X = ['casada', 'mage', 'medu', 'fhisp', 'mhisp', 'foreign', 'alcohol', 'deadkids', 'nprenatal', 'mrace', 'frace', 'fage', 'fedu']
D = "Treated"
y = "Y"

folds = 5

np.random.seed(123)
m_D = GradientBoostingRegressor(n_estimators=100)
D_res2 = df[D] - cross_val_predict(m_D, df[X], df[D], cv=folds)

m_y = GradientBoostingRegressor(n_estimators=100)
y_res2 = df[y] - cross_val_predict(m_y, df[X], df[y], cv=folds)

In [53]:
DML2 = df.assign(**{
    "Y-ML_y(X)": y_res2,
    "Treated-ML_t(X)": D_res2,
})
DML2.head()

,bweight,mmarried,mhisp,fhisp,foreign,alcohol,deadkids,mage,medu,fage,...,fbaby,prenatal1,Y,Treated,casada,m_t_star,m_y_star,m_D_star,Y-ML_y(X),Treated-ML_t(X)
0,3459,married,0,0,0,0,0,24,14,28,...,No,Yes,3459,0,1,-0.086026,47.203835,-0.086026,44.846311,-0.065141
1,3260,notmarried,0,0,1,0,0,20,10,0,...,No,Yes,3260,0,0,-0.315814,277.930277,-0.315814,173.141492,-0.379986
2,3572,married,0,0,1,0,0,22,9,30,...,No,Yes,3572,0,1,-0.186338,196.895284,-0.186338,84.286938,-0.182789
3,2948,married,0,0,0,0,0,26,12,30,...,No,Yes,2948,0,1,-0.156887,-451.977522,-0.156887,-412.680405,-0.171932
4,2410,married,0,0,0,0,0,20,12,21,...,Yes,Yes,2410,0,1,-0.121311,-1040.633348,-0.121311,-1096.367541,-0.070470


In [55]:
FWL_DML2 = smf.ols("y_res2 ~ D_res2", data=DML2).fit()
print(FWL_DML2.summary())

                            OLS Regression Results                            
Dep. Variable:                 y_res2   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     88.64
Date:                qua, 14 ago 2024   Prob (F-statistic):           7.27e-21
Time:                        17:09:05   Log-Likelihood:                -35883.
No. Observations:                4642   AIC:                         7.177e+04
Df Residuals:                    4640   BIC:                         7.178e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8925      8.084      0.110      0.9

Posteriormente, **Chernozhukov, Chetverikov, Demirer, Duflo, Hansen, Newey e Robins (2018)** consideram o seguinte **modelo parcialmente linear**.

$$ Y = \alpha D + g(X) + u D = m(X) + v $$

Onde $Y$ é o resultado, $D$ é o tratamento, $X$ é o vetor de características de controle de alta dimensão.





## Estimação do CATE com DML

Até agora, vimos como o Double/Debiased ML nos permite focar na estimativa do Efeito Médio do Tratamento (ATE), mas ele também pode ser usado para estimar a heterogeneidade dos efeitos do tratamento ou o Efeito Médio Condicional do Tratamento (CATE). Essencialmente, agora estamos dizendo que o parâmetro causal $\tau$ muda dependendo das covariáveis da unidade.

$$
Y_i - {M}_y(X_i) = \tau(X_i) \cdot (T_i - {M}_t(X_i)) + \epsilon_i
$$

Para estimar este modelo, usaremos a mesma versão residualizada de preço e vendas, mas agora vamos interagir os resíduos do preço com as outras covariáveis. Em seguida, podemos ajustar um modelo linear de CATE.

$$
\tilde{Y_i} = \alpha + \beta_1 \tilde{T_i} + \pmb{\beta}_2 \pmb{X_i} \tilde{T_i} + \epsilon_i
$$

Depois de estimarmos tal modelo, para fazer previsões de CATE, usaremos o conjunto de teste randomizado. Como este modelo final é linear, podemos calcular o CATE mecanicamente:

$$
\hat{\mu}(\partial PesoBebê_i, X_i) = M(Fumante=1, X_i) - M(Fumante=0, X_i)
$$

onde $M$ é nosso modelo linear final.


In [81]:
import statsmodels.formula.api as smf

# Estimar o modelo
final_model_cate = smf.ols(formula='y_res1 ~ D_res1*(mage) + casada + mage + medu + fhisp + mhisp + foreign + alcohol + deadkids + nprenatal + mrace + frace + fage + fedu', data=df).fit()
# Calcular o CATE
cate_test = df.assign(cate=final_model_cate.predict(df.assign(D_res1=1)) - final_model_cate.predict(df.assign( D_res1=0)))
cate_test

,bweight,mmarried,mhisp,fhisp,foreign,alcohol,deadkids,mage,medu,fage,...,birthmonth,lbweight,fbaby,prenatal1,Y,Treated,casada,m_D_star,m_y_star,cate
0,3459,married,0,0,0,0,0,24,14,28,...,12,0,No,Yes,3459,0,1,-0.086026,47.203835,-170.004779
1,3260,notmarried,0,0,1,0,0,20,10,0,...,7,0,No,Yes,3260,0,0,-0.315814,277.930277,-134.736065
2,3572,married,0,0,1,0,0,22,9,30,...,3,0,No,Yes,3572,0,1,-0.186338,196.895284,-152.370422
3,2948,married,0,0,0,0,0,26,12,30,...,1,0,No,Yes,2948,0,1,-0.156887,-451.977522,-187.639136
4,2410,married,0,0,0,0,0,20,12,21,...,3,1,Yes,Yes,2410,0,1,-0.121311,-1040.633348,-134.736065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4637,3317,notmarried,0,0,0,0,0,21,12,24,...,10,0,Yes,No,3317,1,0,0.683205,5.917001,-143.553243
4638,3030,married,0,0,0,1,1,30,12,23,...,2,0,No,No,3030,1,1,0.478816,-259.794082,-222.907850
4639,2950,notmarried,0,0,0,0,0,23,12,0,...,5,0,Yes,Yes,2950,0,0,-0.191829,-242.054511,-161.187600
4640,3969,married,0,0,0,0,0,23,12,25,...,2,0,Yes,Yes,3969,0,1,-0.142649,518.911072,-161.187600


## Procedimento de Double/Debiased ML com um Modelo Linear Final

O procedimento de Double/Debiased ML com um modelo linear final já é muito bom, como podemos ver pela curva acima. Mas, talvez possamos fazer ainda melhor. De fato, este é um procedimento muito geral que podemos entender como um meta-aprendizado (*meta-learner*). Nie e Wager o chamaram de R-Learner, como uma forma de reconhecer o trabalho de Robinson (1988) e enfatizar o papel da residualização.

Essa generalização vem do entendimento de que o procedimento de Double/Debiased ML define uma nova função de perda que podemos minimizar da forma que quisermos. A seguir, veremos como fazer isso de maneira muito semelhante ao que vimos antes ao discutir o método de transformação de alvo ou o F-learner.

## Double/Debiased ML Não Paramétrico

O interessante sobre o Double-ML é que ele nos livra de toda a complicação de aprender os parâmetros de incômodo em um modelo causal. Com isso, podemos concentrar toda a nossa atenção em aprender o parâmetro causal de interesse, seja o ATE ou o CATE. No entanto, com a especificação acima, ainda estávamos usando um modelo linear após a residualização por ML, como modelo causal final. Em nosso exemplo, isso significa que estamos assumindo que o preço impacta as vendas de forma linear. Isso provavelmente está OK para uma faixa pequena de preços, mas sabemos pela teoria microeconômica que isso não é necessariamente o caso. Pode ser que, em preços baixos, um aumento unitário no preço reduza a demanda em 2 unidades. Mas então, em preços mais altos, um aumento unitário no preço pode reduzir a demanda em apenas 1 unidade. Isso não é uma relação linear.

Poderíamos usar a teoria microeconômica aqui para especular sobre a forma funcional do desfecho em relação ao tratamento, mas talvez possamos também delegar isso a um modelo de ML. Em outras palavras, deixar a máquina aprender essa forma funcional complicada. Acontece que isso é totalmente possível se fizermos algumas mudanças em nosso algoritmo original de Double/Debiased ML.

Primeiro, começamos exatamente como antes, ortogonalizando o tratamento e o desfecho com previsões cruzadas de um modelo de ML.

Ver:

https://matteocourthoud.github.io/post/double_ml/ 

https://econml.azurewebsites.net/spec/estimation/dml.html#
https://github.com/py-why/EconML/blob/main/notebooks/CATE%20validation.ipynb


### DRML - Doubly Robust Machine Learning


### DDML - Double/Debiased Machine Learning - Chernozhukov et al. (2018)

Também conhecido como o ML duplo/desviado não paramétrico


O bom do Double/Debiased-ML (DDML) é que ele nos livra de todo o incômodo de aprender os parâmetros incômodos em um modelo causal. Com isso, podemos concentrar toda a nossa atenção em aprender o parâmetro causal de interesse, seja o ATE ou o CATE. No entanto, com a especificação acima, ainda estávamos usando um modelo linear após a residualização do ML, como o modelo causal final. Em nosso exemplo, isso significa que estamos assumindo que o preço impacta as vendas linearmente. Isso provavelmente é OK para uma pequena faixa de preços, mas sabemos pela teoria microeconômica que esse não é necessariamente o caso. Pode ser que, a preços baixos, um aumento unitário no preço reduza a demanda em 2 unidades. Mas então, a preços mais altos, um aumento unitário no preço reduza a demanda em apenas 1 unidade. Essa não é uma relação linear.


In [37]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_predict

y = "sales"
T = "price"
X = ["temp", "weekday", "cost"]

debias_m = LGBMRegressor(max_depth=3)

train_pred = train.assign(price_res =  train[T] -
                          cross_val_predict(debias_m, train[X], train[T], cv=5)
                          + train[T].mean()) # add mu_t for visualization. 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000265 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 211
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 3
[LightGBM] [Info] Start training from score 6.143650
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [38]:
y = "sales"
T = "price"
X = ["temp", "weekday", "cost"]

debias_m = LGBMRegressor(max_depth=3)
denoise_m = LGBMRegressor(max_depth=3)

train_pred = train.assign(price_res =  train[T] - cross_val_predict(debias_m, train[X], train[T], cv=5),
                          sales_res =  train[y] - cross_val_predict(denoise_m, train[X], train[y], cv=5))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000026 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 211
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 3
[LightGBM] [Info] Start training from score 6.143650
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

Isso é nada menos que incrível, porque agora podemos chamar isso de função de perda causal . O que significa que, se minimizarmos o quadrado dessa perda, estaremos estimando o valor esperado de
, que é o CATE.

In [41]:
model_final = LGBMRegressor(max_depth=3)
 
# create the weights
w = train_pred["price_res"] ** 2 
 
# create the transformed target
y_star = (train_pred["sales_res"] / train_pred["price_res"])
 
# use a weighted regression ML model to predict the target with the weights.
model_final.fit(X=train[X], y=y_star, sample_weight=w)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 219
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 3
[LightGBM] [Info] Start training from score -3.857895
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMRegressor(max_depth=3)

### Deveríamos usar DD se temos DML?
https://matteocourthoud.github.io/post/did_or_dml/


Em suma:

O principal objetivo do DML é ajustar e remover a variável de confusão de forma que a variável de interesse (tratamento) e o desfecho (resultado) fiquem "ortogonais" ou "independentes".

Abordagem:

DML combina métodos de aprendizado de máquina com técnicas econométricas para estimar efeitos causais.
Utiliza a ideia de "nuisance parameters" (parâmetros de incômodo) que são ajustados e removidos do modelo principal para reduzir o viés.
A técnica geralmente envolve a aplicação de aprendizado de máquina para prever tanto o tratamento quanto o desfecho usando variáveis de confusão, e então os resíduos dessas previsões são utilizados em um segundo estágio para estimar o efeito causal.

Implementação:

Primeiro Estágio: Aplicar modelos de aprendizado de máquina para prever a variável de tratamento e o resultado.
Segundo Estágio: Utilizar os resíduos dessas previsões em um modelo de regressão para estimar o efeito causal.




### Doubly Robust Machine Learning (DRML)

Objetivo:

DRML visa fornecer estimativas consistentes do efeito causal mesmo que uma das duas especificações de modelo (do tratamento ou do desfecho) esteja incorreta.
Abordagem:

A metodologia DRML combina a ponderação de probabilidade inversa (IPW) com métodos de regressão.
Essa abordagem é "duplamente robusta" porque, para a consistência da estimativa, é necessário que apenas um dos modelos - ou o modelo de probabilidade de tratamento (propensity score) ou o modelo de desfecho (outcome model) - esteja corretamente especificado.
Implementação:

Modelagem do Propensity Score: Estimar a probabilidade de cada unidade receber o tratamento.
Modelagem do Desfecho: Estimar o desfecho esperado condicional ao tratamento e às covariáveis.
Combinação dos Modelos: Utilizar uma combinação das estimativas dos dois modelos para obter uma estimativa do efeito causal que é robusta a erros na especificação de um dos modelos.

#### Resumo Comparativo: DML vs DRML

* DML: Foca na remoção de viés através de uma abordagem em dois estágios, utilizando previsões de aprendizado de máquina e resíduos. Busca a ortogonalidade entre o tratamento e o desfecho.
* DRML: Utiliza a ponderação de probabilidade inversa e a modelagem do desfecho para obter estimativas robustas. É duplamente robusta porque requer que apenas um dos dois modelos esteja corretamente especificado.

Ambas as metodologias são úteis em inferência causal e se beneficiam da flexibilidade e poder dos métodos de aprendizado de máquina, mas cada uma tem suas especificidades e contextos de aplicação onde são mais adequadas.